In [104]:
import os
import pandas as pd
from dotenv import load_dotenv
from recsys import utils
from recsys.imdb_parser import etl

load_dotenv()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
bucket = os.getenv('AWS_S3_BUCKET')
storage_options = {
    'key': os.getenv('AWS_ACCESS_KEY'),
    'secret': os.getenv('AWS_SECRET_ACCESS_KEY')
}

In [20]:
from boto3 import client

s3_client = client(
    's3',
    aws_access_key_id=storage_options['key'],
    aws_secret_access_key=storage_options['secret']
)

In [24]:
metadata_b = s3_client.get_object(Bucket=bucket, Key='metadata/metadata.json')['Body'].read()

In [28]:
metadata_b = s3_client.get_object(Bucket=bucket, Key='metadata/metadata.json')['Body'].read()
metadata = pd.read_json(metadata_b, orient='index')

In [36]:
metadata.head()

,main_genre,original_title,genres,poster_url,review_summary,agg_rating,actors,imdb_recommendations,details,boxoffice,reviews_collected_flg
/title/tt0002130/,Adventure,Dante's Inferno,"[Adventure, Drama, Fantasy]",https://m.media-amazon.com/images/M/MV5BNjU1ND...,"{'user_review_num': '36', 'critic_review_num':...","{'avg_rating': '7.0/10', 'num_votes': '2.9K'}","{'1': '/name/nm0660139', '2': '/name/nm0685283...","{'1': '/title/tt0003740/', '2': '/title/tt0001...",{'release_date': ['July 1911 (United States)']...,"{'budget': None, 'boxoffice_gross_domestic': N...",1
/title/tt0003740/,Adventure,Cabiria,"[Adventure, Drama, History]",https://m.media-amazon.com/images/M/MV5BZmVjNz...,"{'user_review_num': '35', 'critic_review_num':...","{'avg_rating': '7.1/10', 'num_votes': '3.6K'}","{'1': '/name/nm0021935', '2': '/name/nm0702894...","{'1': '/title/tt0006864/', '2': '/title/tt0009...","{'release_date': ['May 9, 1914 (United States)...","{'budget': 'ITL 1,000,000 (estimated)', 'boxof...",1
/title/tt0004635/,Action,The Squaw Man,"[Action, Drama, Romance]",https://m.media-amazon.com/images/M/MV5BMjMwOD...,"{'user_review_num': '9', 'critic_review_num': ...","{'avg_rating': '5.7/10', 'num_votes': '979'}","{'1': '/name/nm0267914', '2': '/name/nm0758457...","{'1': '/title/tt0014532/', '2': '/title/tt0006...","{'release_date': ['February 15, 1914 (United S...","{'budget': '$20,000 (estimated)', 'boxoffice_g...",1
/title/tt0004707/,Comedy,Tillie's Punctured Romance,[Comedy],https://m.media-amazon.com/images/M/MV5BMDZjOW...,"{'user_review_num': '41', 'critic_review_num':...","{'avg_rating': '6.3/10', 'num_votes': '3.5K'}","{'1': '/name/nm0000122', '2': '/name/nm0237597...","{'1': '/title/tt0005074/', '2': '/title/tt0006...","{'release_date': ['December 21, 1914 (United S...","{'budget': '$50,000 (estimated)', 'boxoffice_g...",1
/title/tt0004972/,Drama,The Birth of a Nation,"[Drama, History, War]",https://m.media-amazon.com/images/M/MV5BNWZlNj...,"{'user_review_num': '379', 'critic_review_num'...","{'avg_rating': '6.2/10', 'num_votes': '25K'}","{'1': '/name/nm0001273', '2': '/name/nm0550615...","{'1': '/title/tt0006864/', '2': '/title/tt0009...","{'release_date': ['March 21, 1915 (United Stat...","{'budget': '$110,000 (estimated)', 'boxoffice_...",1


In [ ]:
etl.expand_short_form

In [142]:
pd.json_normalize(metadata['review_summary'])

,user_review_num,critic_review_num,metascore
0,36,13,NaN
1,35,26,NaN
2,9,4,NaN
3,41,19,NaN
4,379,79,NaN
...,...,...,...
23579,426,66,35
23580,269,12,NaN
23581,141,83,62
23582,10,5,NaN


In [144]:
n = pd.json_normalize(metadata['review_summary'])
n.head()

,user_review_num,critic_review_num,metascore
0,36,13,NaN
1,35,26,NaN
2,9,4,NaN
3,41,19,NaN
4,379,79,NaN


In [171]:
metadata['agg_rating'].isna().sum()

1

In [183]:
r = etl.split_aggregate_rating(metadata)
r

,main_genre,original_title,genres,poster_url,review_summary,actors,imdb_recommendations,details,boxoffice,reviews_collected_flg,rating,num_votes
/title/tt0002130/,Adventure,Dante's Inferno,"[Adventure, Drama, Fantasy]",https://m.media-amazon.com/images/M/MV5BNjU1ND...,"{'user_review_num': '36', 'critic_review_num':...","{'1': '/name/nm0660139', '2': '/name/nm0685283...","{'1': '/title/tt0003740/', '2': '/title/tt0001...",{'release_date': ['July 1911 (United States)']...,"{'budget': None, 'boxoffice_gross_domestic': N...",1,7.0,2900.0
/title/tt0003740/,Adventure,Cabiria,"[Adventure, Drama, History]",https://m.media-amazon.com/images/M/MV5BZmVjNz...,"{'user_review_num': '35', 'critic_review_num':...","{'1': '/name/nm0021935', '2': '/name/nm0702894...","{'1': '/title/tt0006864/', '2': '/title/tt0009...","{'release_date': ['May 9, 1914 (United States)...","{'budget': 'ITL 1,000,000 (estimated)', 'boxof...",1,7.1,3600.0
/title/tt0004635/,Action,The Squaw Man,"[Action, Drama, Romance]",https://m.media-amazon.com/images/M/MV5BMjMwOD...,"{'user_review_num': '9', 'critic_review_num': ...","{'1': '/name/nm0267914', '2': '/name/nm0758457...","{'1': '/title/tt0014532/', '2': '/title/tt0006...","{'release_date': ['February 15, 1914 (United S...","{'budget': '$20,000 (estimated)', 'boxoffice_g...",1,5.7,979.0
/title/tt0004707/,Comedy,Tillie's Punctured Romance,[Comedy],https://m.media-amazon.com/images/M/MV5BMDZjOW...,"{'user_review_num': '41', 'critic_review_num':...","{'1': '/name/nm0000122', '2': '/name/nm0237597...","{'1': '/title/tt0005074/', '2': '/title/tt0006...","{'release_date': ['December 21, 1914 (United S...","{'budget': '$50,000 (estimated)', 'boxoffice_g...",1,6.3,3500.0
/title/tt0004972/,Drama,The Birth of a Nation,"[Drama, History, War]",https://m.media-amazon.com/images/M/MV5BNWZlNj...,"{'user_review_num': '379', 'critic_review_num'...","{'1': '/name/nm0001273', '2': '/name/nm0550615...","{'1': '/title/tt0006864/', '2': '/title/tt0009...","{'release_date': ['March 21, 1915 (United Stat...","{'budget': '$110,000 (estimated)', 'boxoffice_...",1,6.2,25000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
/title/tt9898858/,Action,Coffee & Kareem,"[Action, Comedy, Crime]",https://m.media-amazon.com/images/M/MV5BNTMyMm...,"{'user_review_num': '426', 'critic_review_num'...","{'1': '/name/nm1159180', '2': '/name/nm0378245...","{'1': '/title/tt8851668/', '2': '/title/tt3317...","{'release_date': ['April 3, 2020 (United State...","{'budget': None, 'boxoffice_gross_domestic': N...",1,5.1,14000.0
/title/tt9900782/,Action,Kaithi,"[Action, Crime, Thriller]",https://m.media-amazon.com/images/M/MV5BOWVlNj...,"{'user_review_num': '269', 'critic_review_num'...","{'1': '/name/nm1912683', '2': '/name/nm1230844...","{'1': '/title/tt9179430/', '2': '/title/tt1057...","{'release_date': ['October 24, 2019 (United St...","{'budget': '₹240,000,000 (estimated)', 'boxoff...",1,8.5,26000.0
/title/tt9907782/,Fantasy,The Cursed,"[Fantasy, Horror, Mystery]",https://m.media-amazon.com/images/M/MV5BMjllNW...,"{'user_review_num': '141', 'critic_review_num'...","{'1': '/name/nm2933542', '2': '/name/nm0717709...","{'1': '/title/tt8296030/', '2': '/title/tt1251...","{'release_date': ['February 18, 2022 (United S...","{'budget': None, 'boxoffice_gross_domestic': '...",1,6.2,8200.0
/title/tt9911196/,Comedy,The Marriage Escape,"[Comedy, Drama]",https://m.media-amazon.com/images/M/MV5BYjNjZm...,"{'user_review_num': '10', 'critic_review_num':...","{'1': '/name/nm0277932', '2': '/name/nm0824373...","{'1': '/title/tt14217100/', '2': '/title/tt613...","{'release_date': ['February 13, 2020 (Netherla...","{'budget': None, 'boxoffice_gross_domestic': N...",1,7.5,3400.0


In [174]:
pd.json_normalize(metadata['agg_rating'], errors='ignore')

AttributeError: 'NoneType' object has no attribute 'items'

In [163]:
etl.split_review_summary(metadata)

,main_genre,original_title,genres,poster_url,agg_rating,actors,imdb_recommendations,details,boxoffice,reviews_collected_flg,user_review_num,critic_review_num,metascore
/title/tt0002130/,Adventure,Dante's Inferno,"[Adventure, Drama, Fantasy]",https://m.media-amazon.com/images/M/MV5BNjU1ND...,"{'avg_rating': '7.0/10', 'num_votes': '2.9K'}","{'1': '/name/nm0660139', '2': '/name/nm0685283...","{'1': '/title/tt0003740/', '2': '/title/tt0001...",{'release_date': ['July 1911 (United States)']...,"{'budget': None, 'boxoffice_gross_domestic': N...",1,36.0,13.0,NaN
/title/tt0003740/,Adventure,Cabiria,"[Adventure, Drama, History]",https://m.media-amazon.com/images/M/MV5BZmVjNz...,"{'avg_rating': '7.1/10', 'num_votes': '3.6K'}","{'1': '/name/nm0021935', '2': '/name/nm0702894...","{'1': '/title/tt0006864/', '2': '/title/tt0009...","{'release_date': ['May 9, 1914 (United States)...","{'budget': 'ITL 1,000,000 (estimated)', 'boxof...",1,35.0,26.0,NaN
/title/tt0004635/,Action,The Squaw Man,"[Action, Drama, Romance]",https://m.media-amazon.com/images/M/MV5BMjMwOD...,"{'avg_rating': '5.7/10', 'num_votes': '979'}","{'1': '/name/nm0267914', '2': '/name/nm0758457...","{'1': '/title/tt0014532/', '2': '/title/tt0006...","{'release_date': ['February 15, 1914 (United S...","{'budget': '$20,000 (estimated)', 'boxoffice_g...",1,9.0,4.0,NaN
/title/tt0004707/,Comedy,Tillie's Punctured Romance,[Comedy],https://m.media-amazon.com/images/M/MV5BMDZjOW...,"{'avg_rating': '6.3/10', 'num_votes': '3.5K'}","{'1': '/name/nm0000122', '2': '/name/nm0237597...","{'1': '/title/tt0005074/', '2': '/title/tt0006...","{'release_date': ['December 21, 1914 (United S...","{'budget': '$50,000 (estimated)', 'boxoffice_g...",1,41.0,19.0,NaN
/title/tt0004972/,Drama,The Birth of a Nation,"[Drama, History, War]",https://m.media-amazon.com/images/M/MV5BNWZlNj...,"{'avg_rating': '6.2/10', 'num_votes': '25K'}","{'1': '/name/nm0001273', '2': '/name/nm0550615...","{'1': '/title/tt0006864/', '2': '/title/tt0009...","{'release_date': ['March 21, 1915 (United Stat...","{'budget': '$110,000 (estimated)', 'boxoffice_...",1,379.0,79.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
/title/tt9898858/,Action,Coffee & Kareem,"[Action, Comedy, Crime]",https://m.media-amazon.com/images/M/MV5BNTMyMm...,"{'avg_rating': '5.1/10', 'num_votes': '14K'}","{'1': '/name/nm1159180', '2': '/name/nm0378245...","{'1': '/title/tt8851668/', '2': '/title/tt3317...","{'release_date': ['April 3, 2020 (United State...","{'budget': None, 'boxoffice_gross_domestic': N...",1,426.0,66.0,35.0
/title/tt9900782/,Action,Kaithi,"[Action, Crime, Thriller]",https://m.media-amazon.com/images/M/MV5BOWVlNj...,"{'avg_rating': '8.5/10', 'num_votes': '26K'}","{'1': '/name/nm1912683', '2': '/name/nm1230844...","{'1': '/title/tt9179430/', '2': '/title/tt1057...","{'release_date': ['October 24, 2019 (United St...","{'budget': '₹240,000,000 (estimated)', 'boxoff...",1,269.0,12.0,NaN
/title/tt9907782/,Fantasy,The Cursed,"[Fantasy, Horror, Mystery]",https://m.media-amazon.com/images/M/MV5BMjllNW...,"{'avg_rating': '6.2/10', 'num_votes': '8.2K'}","{'1': '/name/nm2933542', '2': '/name/nm0717709...","{'1': '/title/tt8296030/', '2': '/title/tt1251...","{'release_date': ['February 18, 2022 (United S...","{'budget': None, 'boxoffice_gross_domestic': '...",1,141.0,83.0,62.0
/title/tt9911196/,Comedy,The Marriage Escape,"[Comedy, Drama]",https://m.media-amazon.com/images/M/MV5BYjNjZm...,"{'avg_rating': '7.5/10', 'num_votes': '3.4K'}","{'1': '/name/nm0277932', '2': '/name/nm0824373...","{'1': '/title/tt14217100/', '2': '/title/tt613...","{'release_date': ['February 13, 2020 (Netherla...","{'budget': None, 'boxoffice_gross_domestic': N...",1,10.0,5.0,NaN


In [126]:
metadata['boxoffice'].iloc[0]

{'budget': None,
 'boxoffice_gross_domestic': None,
 'boxoffice_gross_opening': None,
 'boxoffice_gross_worldwide': None}

In [127]:
metadata['boxoffice'].iloc[1]

{'budget': 'ITL\xa01,000,000 (estimated)',
 'boxoffice_gross_domestic': None,
 'boxoffice_gross_opening': None,
 'boxoffice_gross_worldwide': None}

In [134]:
metadata['boxoffice'].iloc[100]

{'budget': None,
 'boxoffice_gross_domestic': None,
 'boxoffice_gross_opening': None,
 'boxoffice_gross_worldwide': '$4,347'}

In [125]:
etl.normalize(metadata, 'imdb_recommendations').loc[0, 1]

'/title/tt0003740/'

In [117]:
for i in metadata['review_summary'].values:
    if 'metascore' in i:
        print(i)
        break

{'user_review_num': '125', 'critic_review_num': '76', 'metascore': '99'}


In [115]:
metadata['review_summary'].iloc[4]

{'user_review_num': '379', 'critic_review_num': '79'}

In [108]:
metadata['actors'].iloc[0]

{'1': '/name/nm0660139',
 '2': '/name/nm0685283',
 '3': '/name/nm0209738',
 '4': '/name/nm3942815',
 '5': '/name/nm1375863',
 '6': '/name/nm1375916',
 '7': '/name/nm1374534'}

In [102]:
pd.json_normalize(metadata['imdb_recommendations'])

,1,2,3,4,5,6,7,8,9,10,11,12
0,/title/tt0003740/,/title/tt0001223/,/title/tt0001527/,/title/tt0003419/,/title/tt0003424/,/title/tt0014826/,/title/tt0026262/,/title/tt0000832/,/title/tt0017048/,/title/tt0479008/,/title/tt0210675/,/title/tt0006864/
1,/title/tt0006864/,/title/tt0009968/,/title/tt0004972/,/title/tt0002130/,/title/tt0003419/,/title/tt0002445/,/title/tt0002844/,/title/tt0000832/,/title/tt0011841/,/title/tt0001105/,/title/tt0006206/,/title/tt0001527/
2,/title/tt0014532/,/title/tt0006886/,/title/tt0210824/,/title/tt0003740/,/title/tt0001527/,/title/tt0003424/,/title/tt0004066/,/title/tt0004465/,/title/tt0003014/,/title/tt0019824/,/title/tt0022428/,/title/tt0009650/
3,/title/tt0005074/,/title/tt0006177/,/title/tt0005077/,/title/tt0004100/,/title/tt0004936/,/title/tt0004288/,/title/tt0004194/,/title/tt0004546/,/title/tt0004395/,/title/tt0004011/,/title/tt0006313/,/title/tt0006309/
4,/title/tt0006864/,/title/tt0009968/,/title/tt0000439/,/title/tt4196450/,/title/tt0011841/,/title/tt0003740/,/title/tt0006664/,/title/tt0012532/,/title/tt0000417/,/title/tt0015648/,/title/tt0010323/,/title/tt0015881/
...,...,...,...,...,...,...,...,...,...,...,...,...
23579,/title/tt8851668/,/title/tt3317234/,/title/tt1542768/,/title/tt9619798/,/title/tt7734218/,/title/tt4769836/,/title/tt8629748/,/title/tt6273924/,/title/tt8976710/,/title/tt7256866/,/title/tt10682266/,/title/tt7517330/
23580,/title/tt9179430/,/title/tt10579952/,/title/tt6148156/,/title/tt10189514/,/title/tt9477520/,/title/tt7060344/,/title/tt5559528/,/title/tt9851854/,/title/tt13391710/,/title/tt15097216/,/title/tt10698680/,/title/tt11580854/
23581,/title/tt8296030/,/title/tt12519030/,/title/tt13560574/,/title/tt13872248/,/title/tt10406596/,/title/tt14550642/,/title/tt14905650/,/title/tt12004038/,/title/tt7550014/,/title/tt17640714/,/title/tt1845838/,/title/tt11138512/
23582,/title/tt14217100/,/title/tt6132250/,/title/tt8639136/,/title/tt6851410/,/title/tt10704730/,/title/tt5571814/,/title/tt13323330/,/title/tt4568328/,/title/tt10521092/,/title/tt8458582/,/title/tt15012846/,/title/tt8095076/


In [101]:
pd.json_normalize(metadata['actors'])

,1,2,3,4,5,6,7,8,9,10
0,/name/nm0660139,/name/nm0685283,/name/nm0209738,/name/nm3942815,/name/nm1375863,/name/nm1375916,/name/nm1374534,NaN,NaN,NaN
1,/name/nm0021935,/name/nm0702894,/name/nm0656034,/name/nm0146028,/name/nm0544842,/name/nm0856422,/name/nm0610618,/name/nm0224122,/name/nm0889527,/name/nm0202688
2,/name/nm0267914,/name/nm0758457,/name/nm0455612,/name/nm0277317,/name/nm0298243,/name/nm0934969,/name/nm0461620,/name/nm0598767,/name/nm0211329,/name/nm0350628
3,/name/nm0000122,/name/nm0237597,/name/nm0635667,/name/nm0841501,/name/nm0071658,/name/nm0174682,/name/nm0016744,/name/nm0020914,/name/nm0071616,/name/nm0096094
4,/name/nm0001273,/name/nm0550615,/name/nm0910400,/name/nm0178270,/name/nm0017488,/name/nm0507635,/name/nm0797094,/name/nm0519227,/name/nm0366008,/name/nm0717468
...,...,...,...,...,...,...,...,...,...,...
23579,/name/nm1159180,/name/nm0378245,/name/nm10067359,/name/nm2365811,/name/nm0498165,/name/nm0004979,/name/nm4392718,/name/nm2285290,/name/nm0396369,/name/nm3552604
23580,/name/nm1912683,/name/nm1230844,/name/nm10183124,/name/nm6998719,/name/nm6649988,/name/nm5782889,/name/nm6030469,/name/nm7639503,/name/nm7130871,/name/nm7645843
23581,/name/nm2933542,/name/nm0717709,/name/nm0677944,/name/nm3646923,/name/nm0079451,/name/nm0101400,/name/nm0475336,/name/nm6076321,/name/nm10498203,/name/nm7748240
23582,/name/nm0277932,/name/nm0824373,/name/nm3362584,/name/nm2558112,/name/nm10877188,/name/nm10877187,/name/nm1135487,/name/nm10877189,/name/nm2099145,/name/nm0119870


In [72]:
mapping = {'K': 'e3', 'M': 'e6'}
review_summary = metadata[['original_title', 'review_summary']].copy(deep=True)
review_summary[['user_review_num', 'critic_review_num', 'metascore']] = review_summary['review_summary'].apply(pd.Series).replace('.', '')
review_summary = review_summary.drop('review_summary', axis=1)
review_summary['user_review_num'] = review_summary['user_review_num'].str.replace('K', 'e3').astype(float)
review_summary.head()

,original_title,user_review_num,critic_review_num,metascore
/title/tt0002130/,Dante's Inferno,36.0,13,NaN
/title/tt0003740/,Cabiria,35.0,26,NaN
/title/tt0004635/,The Squaw Man,9.0,4,NaN
/title/tt0004707/,Tillie's Punctured Romance,41.0,19,NaN
/title/tt0004972/,The Birth of a Nation,379.0,79,NaN


In [70]:
review_summary[review_summary['user_review_num'] == '4.7K']

,original_title,user_review_num,critic_review_num,metascore
/title/tt9531772/,Sooryavanshi,4.7K,15,NaN


In [74]:
review_summary['user_review_num'].sum()

3907296.0

In [86]:
top_n = 1000
top_movies = review_summary['user_review_num'].sort_values(ascending=False).dropna().index[:top_n]
top_movies

Index(['/title/tt7286456/', '/title/tt0111161/', '/title/tt4154796/',
       '/title/tt0468569/', '/title/tt12361974/', '/title/tt2527338/',
       '/title/tt8110330/', '/title/tt10811166/', '/title/tt7126948/',
       '/title/tt4154664/',
       ...
       '/title/tt8983202/', '/title/tt0112641/', '/title/tt3567288/',
       '/title/tt11314148/', '/title/tt2094766/', '/title/tt7886848/',
       '/title/tt10065694/', '/title/tt1527186/', '/title/tt0385752/',
       '/title/tt7518786/'],
      dtype='object', length=1000)

In [90]:
from tqdm.notebook import tqdm

In [95]:
review_dfs[0].iloc[0]

Unnamed: 0                                                     0
id                                             /title/tt7286456/
text           Truly a masterpiece, The Best Hollywood film o...
rating                                                      10.0
date                                            1 September 2019
title                                 Perfect in every aspect.\n
author                             /user/ur68129834/?ref_=tt_urv
helpfulness    \n                    3,088 out of 4,018 found...
Name: 0, dtype: object

In [105]:
etl.normalize(review_dfs[0].iloc[0])

AttributeError: 'Series' object has no attribute 'columns'

In [91]:
review_dfs = []
for title_id in tqdm(top_movies):
    review_file = title_id.split('/title/')[1][:-1] + '.csv'
    title_reviews_file = os.path.join('s3://', bucket, 'reviews', review_file)
    some_reviews = pd.read_csv(title_reviews_file, storage_options=storage_options)
    review_dfs.append(some_reviews)

  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [75]:
%%time
metadata_file = os.path.join('s3://', bucket, 'metadata/metadata.json')
metadata = pd.read_json(metadata_file, orient='index', storage_options=storage_options)

CPU times: user 1.18 s, sys: 147 ms, total: 1.33 s
Wall time: 20.5 s


In [284]:
pd.read_json(os.path.join('s3://', bucket, 'metadata/metadata11.json'), orient='index', storage_options=storage_options)

FileNotFoundError: movie-on-friday/metadata/metadata11.json

In [78]:
metadata.index

Index(['/title/tt0002130/', '/title/tt0003740/', '/title/tt0004635/',
       '/title/tt0004707/', '/title/tt0004972/', '/title/tt0006206/',
       '/title/tt0006780/', '/title/tt0006864/', '/title/tt0009893/',
       '/title/tt0009900/',
       ...
       '/title/tt9877170/', '/title/tt9883996/', '/title/tt9893250/',
       '/title/tt9894440/', '/title/tt9894470/', '/title/tt9898858/',
       '/title/tt9900782/', '/title/tt9907782/', '/title/tt9911196/',
       '/title/tt9916362/'],
      dtype='object', length=23584)

In [79]:
metadata

,main_genre,original_title,genres,poster_url,review_summary,agg_rating,actors,imdb_recommendations,details,boxoffice,reviews_collected_flg
/title/tt0002130/,Adventure,Dante's Inferno,"[Adventure, Drama, Fantasy]",https://m.media-amazon.com/images/M/MV5BNjU1ND...,"{'user_review_num': '36', 'critic_review_num':...","{'avg_rating': '7.0/10', 'num_votes': '2.9K'}","{'1': '/name/nm0660139', '2': '/name/nm0685283...","{'1': '/title/tt0003740/', '2': '/title/tt0001...",{'release_date': ['July 1911 (United States)']...,"{'budget': None, 'boxoffice_gross_domestic': N...",1
/title/tt0003740/,Adventure,Cabiria,"[Adventure, Drama, History]",https://m.media-amazon.com/images/M/MV5BZmVjNz...,"{'user_review_num': '35', 'critic_review_num':...","{'avg_rating': '7.1/10', 'num_votes': '3.6K'}","{'1': '/name/nm0021935', '2': '/name/nm0702894...","{'1': '/title/tt0006864/', '2': '/title/tt0009...","{'release_date': ['May 9, 1914 (United States)...","{'budget': 'ITL 1,000,000 (estimated)', 'boxof...",1
/title/tt0004635/,Action,The Squaw Man,"[Action, Drama, Romance]",https://m.media-amazon.com/images/M/MV5BMjMwOD...,"{'user_review_num': '9', 'critic_review_num': ...","{'avg_rating': '5.7/10', 'num_votes': '979'}","{'1': '/name/nm0267914', '2': '/name/nm0758457...","{'1': '/title/tt0014532/', '2': '/title/tt0006...","{'release_date': ['February 15, 1914 (United S...","{'budget': '$20,000 (estimated)', 'boxoffice_g...",1
/title/tt0004707/,Comedy,Tillie's Punctured Romance,[Comedy],https://m.media-amazon.com/images/M/MV5BMDZjOW...,"{'user_review_num': '41', 'critic_review_num':...","{'avg_rating': '6.3/10', 'num_votes': '3.5K'}","{'1': '/name/nm0000122', '2': '/name/nm0237597...","{'1': '/title/tt0005074/', '2': '/title/tt0006...","{'release_date': ['December 21, 1914 (United S...","{'budget': '$50,000 (estimated)', 'boxoffice_g...",1
/title/tt0004972/,Drama,The Birth of a Nation,"[Drama, History, War]",https://m.media-amazon.com/images/M/MV5BNWZlNj...,"{'user_review_num': '379', 'critic_review_num'...","{'avg_rating': '6.2/10', 'num_votes': '25K'}","{'1': '/name/nm0001273', '2': '/name/nm0550615...","{'1': '/title/tt0006864/', '2': '/title/tt0009...","{'release_date': ['March 21, 1915 (United Stat...","{'budget': '$110,000 (estimated)', 'boxoffice_...",1
...,...,...,...,...,...,...,...,...,...,...,...
/title/tt9898858/,Action,Coffee & Kareem,"[Action, Comedy, Crime]",https://m.media-amazon.com/images/M/MV5BNTMyMm...,"{'user_review_num': '426', 'critic_review_num'...","{'avg_rating': '5.1/10', 'num_votes': '14K'}","{'1': '/name/nm1159180', '2': '/name/nm0378245...","{'1': '/title/tt8851668/', '2': '/title/tt3317...","{'release_date': ['April 3, 2020 (United State...","{'budget': None, 'boxoffice_gross_domestic': N...",1
/title/tt9900782/,Action,Kaithi,"[Action, Crime, Thriller]",https://m.media-amazon.com/images/M/MV5BOWVlNj...,"{'user_review_num': '269', 'critic_review_num'...","{'avg_rating': '8.5/10', 'num_votes': '26K'}","{'1': '/name/nm1912683', '2': '/name/nm1230844...","{'1': '/title/tt9179430/', '2': '/title/tt1057...","{'release_date': ['October 24, 2019 (United St...","{'budget': '₹240,000,000 (estimated)', 'boxoff...",1
/title/tt9907782/,Fantasy,The Cursed,"[Fantasy, Horror, Mystery]",https://m.media-amazon.com/images/M/MV5BMjllNW...,"{'user_review_num': '141', 'critic_review_num'...","{'avg_rating': '6.2/10', 'num_votes': '8.2K'}","{'1': '/name/nm2933542', '2': '/name/nm0717709...","{'1': '/title/tt8296030/', '2': '/title/tt1251...","{'release_date': ['February 18, 2022 (United S...","{'budget': None, 'boxoffice_gross_domestic': '...",1
/title/tt9911196/,Comedy,The Marriage Escape,"[Comedy, Drama]",https://m.media-amazon.com/images/M/MV5BYjNjZm...,"{'user_review_num': '10', 'critic_review_num':...","{'avg_rating': '7.5/10', 'num_votes': '3.4K'}","{'1': '/name/nm0277932', '2': '/name/nm0824373...","{'1': '/title/tt14217100/', '2': '/title/tt613...","{'release_date': ['February 13, 2020 (Netherla...","{'budget': None, 'boxoffice_gross_domes

In [266]:
h = metadata['original_title'].apply(hash).mod(10).reset_index().groupby('original_title').agg(set)
h

,index
original_title,
0,"{/title/tt0079322/, /title/tt0069897/, /title/..."
1,"{/title/tt0063663/, /title/tt0098253/, /title/..."
2,"{/title/tt0070510/, /title/tt0484111/, /title/..."
3,"{/title/tt0027438/, /title/tt0294870/, /title/..."
4,"{/title/tt2368254/, /title/tt3281548/, /title/..."
5,"{/title/tt0078077/, /title/tt0109417/, /title/..."
6,"{/title/tt0074705/, /title/tt5493944/, /title/..."
7,"{/title/tt0120241/, /title/tt0100506/, /title/..."
8,"{/title/tt0089767/, /title/tt2987732/, /title/..."


In [271]:
s = '/title/tt0079322/'
s

'/title/tt0079322/'

In [277]:
s[7:-1]

'tt0079322'

In [269]:
h.at[0, 'index']

{'/title/tt0079322/',
 '/title/tt0069897/',
 '/title/tt0113799/',
 '/title/tt0090567/',
 '/title/tt0322023/',
 '/title/tt1993391/',
 '/title/tt0056736/',
 '/title/tt0112767/',
 '/title/tt15004136/',
 '/title/tt3469592/',
 '/title/tt0075790/',
 '/title/tt5862312/',
 '/title/tt0155711/',
 '/title/tt0206921/',
 '/title/tt0424430/',
 '/title/tt7268738/',
 '/title/tt0078350/',
 '/title/tt2716382/',
 '/title/tt0065106/',
 '/title/tt0036696/',
 '/title/tt0364376/',
 '/title/tt0084737/',
 '/title/tt1181795/',
 '/title/tt1017464/',
 '/title/tt0061024/',
 '/title/tt0449869/',
 '/title/tt2677064/',
 '/title/tt0110006/',
 '/title/tt0280949/',
 '/title/tt0322645/',
 '/title/tt1667307/',
 '/title/tt13249596/',
 '/title/tt1305583/',
 '/title/tt0077696/',
 '/title/tt1190536/',
 '/title/tt0087258/',
 '/title/tt0109099/',
 '/title/tt0109771/',
 '/title/tt0395173/',
 '/title/tt3625136/',
 '/title/tt9213244/',
 '/title/tt2866708/',
 '/title/tt0055318/',
 '/title/tt0374330/',
 '/title/tt0082250/',
 '/title

In [234]:
metadata

,main_genre,original_title,genres,poster_url,review_summary,agg_rating,actors,imdb_recommendations,details,boxoffice,reviews_collected_flg
/title/tt0002130/,Adventure,Dante's Inferno,"[Adventure, Drama, Fantasy]",https://m.media-amazon.com/images/M/MV5BNjU1ND...,"{'user_review_num': '36', 'critic_review_num':...","{'avg_rating': '7.0/10', 'num_votes': '2.9K'}","{'1': '/name/nm0660139', '2': '/name/nm0685283...","{'1': '/title/tt0003740/', '2': '/title/tt0001...",{'release_date': ['July 1911 (United States)']...,"{'budget': None, 'boxoffice_gross_domestic': N...",1
/title/tt0003740/,Adventure,Cabiria,"[Adventure, Drama, History]",https://m.media-amazon.com/images/M/MV5BZmVjNz...,"{'user_review_num': '35', 'critic_review_num':...","{'avg_rating': '7.1/10', 'num_votes': '3.6K'}","{'1': '/name/nm0021935', '2': '/name/nm0702894...","{'1': '/title/tt0006864/', '2': '/title/tt0009...","{'release_date': ['May 9, 1914 (United States)...","{'budget': 'ITL 1,000,000 (estimated)', 'boxof...",1
/title/tt0004635/,Action,The Squaw Man,"[Action, Drama, Romance]",https://m.media-amazon.com/images/M/MV5BMjMwOD...,"{'user_review_num': '9', 'critic_review_num': ...","{'avg_rating': '5.7/10', 'num_votes': '979'}","{'1': '/name/nm0267914', '2': '/name/nm0758457...","{'1': '/title/tt0014532/', '2': '/title/tt0006...","{'release_date': ['February 15, 1914 (United S...","{'budget': '$20,000 (estimated)', 'boxoffice_g...",1
/title/tt0004707/,Comedy,Tillie's Punctured Romance,[Comedy],https://m.media-amazon.com/images/M/MV5BMDZjOW...,"{'user_review_num': '41', 'critic_review_num':...","{'avg_rating': '6.3/10', 'num_votes': '3.5K'}","{'1': '/name/nm0000122', '2': '/name/nm0237597...","{'1': '/title/tt0005074/', '2': '/title/tt0006...","{'release_date': ['December 21, 1914 (United S...","{'budget': '$50,000 (estimated)', 'boxoffice_g...",1
/title/tt0004972/,Drama,The Birth of a Nation,"[Drama, History, War]",https://m.media-amazon.com/images/M/MV5BNWZlNj...,"{'user_review_num': '379', 'critic_review_num'...","{'avg_rating': '6.2/10', 'num_votes': '25K'}","{'1': '/name/nm0001273', '2': '/name/nm0550615...","{'1': '/title/tt0006864/', '2': '/title/tt0009...","{'release_date': ['March 21, 1915 (United Stat...","{'budget': '$110,000 (estimated)', 'boxoffice_...",1
...,...,...,...,...,...,...,...,...,...,...,...
/title/tt9898858/,Action,Coffee & Kareem,"[Action, Comedy, Crime]",https://m.media-amazon.com/images/M/MV5BNTMyMm...,"{'user_review_num': '426', 'critic_review_num'...","{'avg_rating': '5.1/10', 'num_votes': '14K'}","{'1': '/name/nm1159180', '2': '/name/nm0378245...","{'1': '/title/tt8851668/', '2': '/title/tt3317...","{'release_date': ['April 3, 2020 (United State...","{'budget': None, 'boxoffice_gross_domestic': N...",1
/title/tt9900782/,Action,Kaithi,"[Action, Crime, Thriller]",https://m.media-amazon.com/images/M/MV5BOWVlNj...,"{'user_review_num': '269', 'critic_review_num'...","{'avg_rating': '8.5/10', 'num_votes': '26K'}","{'1': '/name/nm1912683', '2': '/name/nm1230844...","{'1': '/title/tt9179430/', '2': '/title/tt1057...","{'release_date': ['October 24, 2019 (United St...","{'budget': '₹240,000,000 (estimated)', 'boxoff...",1
/title/tt9907782/,Fantasy,The Cursed,"[Fantasy, Horror, Mystery]",https://m.media-amazon.com/images/M/MV5BMjllNW...,"{'user_review_num': '141', 'critic_review_num'...","{'avg_rating': '6.2/10', 'num_votes': '8.2K'}","{'1': '/name/nm2933542', '2': '/name/nm0717709...","{'1': '/title/tt8296030/', '2': '/title/tt1251...","{'release_date': ['February 18, 2022 (United S...","{'budget': None, 'boxoffice_gross_domestic': '...",1
/title/tt9911196/,Comedy,The Marriage Escape,"[Comedy, Drama]",https://m.media-amazon.com/images/M/MV5BYjNjZm...,"{'user_review_num': '10', 'critic_review_num':...","{'avg_rating': '7.5/10', 'num_votes': '3.4K'}","{'1': '/name/nm0277932', '2': '/name/nm0824373...","{'1': '/title/tt14217100/', '2': '/title/tt613...","{'release_date': ['February 13, 2020 (Netherla...","{'budget': None, 'boxoffice_gross_domes

In [76]:
for title in metadata.index:
title_reviews_file = os.path.join('s3://', bucket, 'reviews/tt0021884.csv')
some_reviews = pd.read_csv(title_reviews_file, storage_options=storage_options)
some_reviews.head()

,Unnamed: 0,id,text,rating,date,title,author,helpfulness
0,0,/title/tt0021884/,"'Frankenstein', like Todd Browning's 'Dracula'...",NaN,16 April 2003,Wow! What a movie! A horror classic and still...,/user/ur0381265/?ref_=tt_urv,\n 15 out of 15 found this ...
1,1,/title/tt0021884/,"""We are about to unfold the story of Frankenst...",10.0,7 August 2009,Classic Monster Movie\n,/user/ur19664654/?ref_=tt_urv,\n 23 out of 23 found this ...
2,2,/title/tt0021884/,Of all the great Universal horror classics and...,10.0,25 January 2008,Slaughter of the Innocents\n,/user/ur2483625/?ref_=tt_urv,\n 13 out of 14 found this ...
3,3,/title/tt0021884/,After having been kicked out of school for his...,10.0,31 January 2005,Whale's First Masterpiece\n,/user/ur0139258/?ref_=tt_urv,\n 81 out of 97 found this ...
4,4,/title/tt0021884/,Even though in 1931 this movie was already a r...,10.0,26 November 2003,Karloff's most memorable\n,/user/ur2626332/?ref_=tt_urv,\n 15 out of 16 found this ...


In [282]:
pd.concat([some_reviews, some_reviews])

,Unnamed: 0,id,text,rating,date,title,author,helpfulness
0,0,/title/tt1160419/,It's been amazing being back in cinemas after ...,10.0,23 October 2021,"My movie of 2021, so far.\n",/user/ur15311310/?ref_=tt_urv,"\n 1,313 out of 2,021 found..."
1,1,/title/tt1160419/,Denis Villeneuve has accomplished what was con...,9.0,13 September 2021,DUNE - A Great Modern Sci-Fi\n,/user/ur0406567/?ref_=tt_urv,"\n 1,144 out of 1,773 found..."
2,2,/title/tt1160419/,It's been some time since I've read the Frank ...,9.0,22 October 2021,"""He shall know your ways as though born to th...",/user/ur2707735/?ref_=tt_urv,\n 567 out of 942 found thi...
3,3,/title/tt1160419/,There is probably no point in warning about sp...,4.0,23 October 2021,Dune- as Endless as a Desert.\n,/user/ur5348015/?ref_=tt_urv,\n 420 out of 709 found thi...
4,4,/title/tt1160419/,I feel I should start by saying I have minimal...,5.0,29 October 2021,Duned from the Start\n,/user/ur102601261/?ref_=tt_urv,\n 111 out of 178 found thi...
...,...,...,...,...,...,...,...,...
5599,5599,/title/tt1160419/,Read Book before watching the movie after the ...,7.0,6 November 2021,Epic but\n,/user/ur77835361/,\n 0 out of 7 found this he...
5600,5600,/title/tt1160419/,I loved the movie right up to where they ended...,8.0,25 October 2021,Good film\n,/user/ur84090370/,\n 0 out of 0 found this he...
5601,5601,/title/tt1160419/,"I liked the movie, but they spoke mostly in wh...",7.0,27 October 2021,"Hard to hear what they're saying, great casti...",/user/ur93516241/,\n 0 out of 6 found this he...
5602,5602,/title/tt1160419/,"Let me say Dune is my favorite syfi movie, but...",5.0,24 October 2021,Disappointed\n,/user/ur97358399/,\n 0 out of 1 found this he...


In [281]:
some_reviews.to_parquet(os.path.join('s3://', bucket, 'test_file.parquet'), storage_options=storage_options)

In [238]:
etl.ReviewsETL.transform(some_reviews).to_csv('here.csv', compression='gzip')

In [241]:
4e6 / 5604 * 7

4996.431120628124

In [239]:
some_reviews.shape

(5604, 8)

In [214]:
some_reviews.memory_usage(deep=True).sum() / 1024 / 1024

7.596530914306641

In [209]:
etl.split_helpfulness_col(some_reviews).dtypes

[autoreload of recsys.imdb_parser.etl failed: Traceback (most recent call last):
  File "/Users/imilyutin/miniconda3/envs/movie_recsys/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/Users/imilyutin/miniconda3/envs/movie_recsys/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/Users/imilyutin/miniconda3/envs/movie_recsys/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 851, in exec_module
  File "<frozen importlib._bootstrap_external>", line 988, in get_code
  File "<frozen importlib._bootstrap_external>", line 918, in source_to_code
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/Users/imilyutin/projects/movie_recommender_system/re

Unnamed: 0       int64
id              object
text            object
rating         float64
date            object
title           object
author          object
upvotes          int64
total_votes      int64
dtype: object

In [190]:
some_reviews.dtypes

Unnamed: 0       int64
id              object
text            object
rating         float64
date            object
title           object
author          object
helpfulness     object
dtype: object

In [188]:
s = 'abc\n'
s.rstrip('\n')

'abc'

In [185]:
some_reviews.shape

(5604, 8)

In [186]:
some_reviews.memory_usage(deep=True) / 1024 / 1024 / 5604 * 4e6

Index             0.087131
Unnamed: 0       30.517578
id              282.287598
text           3259.999168
rating           30.517578
date            274.961800
title           335.984703
author          283.757931
helpfulness     924.107009
dtype: float64

In [10]:
some_reviews.memory_usage(deep=True).sum() / 1024 / 1024 * 23000

29241.442680358887